# IMPORTING ALL DEPENDANCIES I NEED FOR THIS PROJECT

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import statsmodels.graphics.correlation as sgc
from statsmodels.graphics.gofplots import qqplot
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# CONNECTION TO MY DATABASE ON POSTGRES

This is the connection link to my database on postgreSQL, the actual connection function is on the file **db_connect.py**

In [ ]:
# Import necessary packages
import pandas as pd
import sys
sys.path.append('..')  # Go up one level to the root folder
from db_connect import connect_to_db

# Step 1: Connect to the database
conn = connect_to_db()

# Step 2: Create a cursor and run a query
cursor = conn.cursor()
query = "SELECT * FROM airbnbs_nairobi.listing_data_yearly;"
cursor.execute(query)

# Step 3: Fetch results and convert to a DataFrame
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Step 4: Display the data
print("Connection successful! Previewing data:")
display(df.head())

# Data Exploration an attempt at understanding my data

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
msno.matrix(df)

# Pricing & Revenue Analysis

### What's the correlation between listing type and average nightly rate?
For this analysis, room type is used as the primary classification variable, as it provides a more general and consistent categorization of listings. In contrast, listing type contains a large number of highly specific categories, which can introduce unnecessary complexity and reduce comparability across observations.

In [ ]:
listing_type = df['listing_type'].unique()
len(listing_type)

In [ ]:
room_type = df['room_type'].unique()
len(room_type)
room_type

While **listing types** consist of **29 unique categories** across the dataset, **room types** are limited to **three broad and representative categories** that better capture the nature of the accommodation. These include **Private Room**, **Entire Home/Apartment**, and **Hotel Room**, the latter of which appears only once in the dataset.

#### Distribution of Room Type across the entire dataset

In [ ]:
room_type_count = df['room_type'].value_counts()
plt.figure(figsize=(10,8))
plt.pie(room_type_count, labels=room_type_count.index, colors=("#0FB9B9", "#E29015", "#F3013E"), autopct='%1.1f%%', startangle=90, textprops={'fontsize':11})
plt.title("Room Type Distribution Across the Dataset", fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()